## Renewables.ninja Wind (Point API) - 52.517, 13.389 - Version: 1.1 - License: https://creativecommons.org/licenses/by-nc/4.0/ - Reference: https://doi.org/10.1016/j.energy.2016.08.068
### Units: time in UTC, local_time in Europe/Berlin, electricity in kW
### {"units": {"time": "UTC", "local_time": "Europe/Berlin", "electricity": "kW"}, "params": {"local_time": true, "header": true, "lat": "52.5170365", "lon": "13.3888599", "date_from": "2019-01-01", "date_to": "2019-12-31", "dataset": "merra2", "capacity": "1", "height": "80", "turbine": "Vestas V90 2000", "raw": false}}

# Dataset refactored for turbine: Automaxx WINDMILL 1500w wind turbine [1 unit]
## Wind Turbine capacity = 1kW --> wind farm for the simulation (8 turbines) = 12kW

In [1]:
%%capture
!pip install plotly==5.9.0

In [2]:
#Cloning repository and changing directory
!git clone https://github.com/francescomaldonato/RL_VPP_Thesis.git
%cd RL_VPP_Thesis/data/data_training/
%ls

In [4]:
#Moving to the current directory
current_folder = ''

input_folder = current_folder + 'raw_datasets/'
output_folder = current_folder + 'scenario_datasets/'

In [5]:
import numpy as np
import pandas as pd
import plotly.express as px

Data pre-process

In [6]:
wt_data = pd.read_csv(input_folder + 'ninja_wind_52.5170_13.3889_corrected_2019_MERRA_Berlin.csv')
wt_data = wt_data.drop(['local_time'], axis=1)
wt_data['time'] = pd.to_datetime(wt_data['time'])

index = len(wt_data['time'])
timestamp = pd.to_datetime("2020-01-01 00:00:00", format="%Y-%m-%d %H:%M:%S")
last_value = wt_data['electricity'][index-1]
new_row = pd.DataFrame([[timestamp, last_value]], columns=["time",'electricity'], index=[index])
wt_data = pd.concat([wt_data, pd.DataFrame(new_row)], ignore_index=False)
wt_data.rename(columns = {'electricity':'wind_power'}, inplace = True)

#Scaling capacity of WT
wt_data['wind_power'] = wt_data['wind_power']*12
wt_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8761 entries, 0 to 8760
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   time        8761 non-null   datetime64[ns]
 1   wind_power  8761 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 205.3 KB


Resampling dataset

In [7]:
wt_data_15 = wt_data.resample('15min', on='time').agg({'time':'min','wind_power':'min'})
wt_data_15['time'] = wt_data_15.index.values
wt_data_15['wind_power'].fillna(method='ffill', inplace=True)
wt_data_15 = wt_data_15.drop(['time'], axis=1)

wt_data_ = wt_data_15.loc['2019-01-01 00:00:00':'2020-01-01 00:00:00']
wt_data_.tail()

,wind_power
time,
2019-12-31 23:00:00,1.632
2019-12-31 23:15:00,1.632
2019-12-31 23:30:00,1.632
2019-12-31 23:45:00,1.632
2020-01-01 00:00:00,1.632


Saving and plotting data

In [8]:
wt_data_csv = wt_data_.to_csv(output_folder + 'WT_load_2019_profile.csv', index = True)
#wt_data_.iplot(title='WT wind power', yTitle='kW', colors='#3aa2cd')
px.line(wt_data_["wind_power"], title=('WT wind power'), color_discrete_sequence=["#3aa2cd"]).update_layout(yaxis_title="kW", xaxis_rangeslider_visible=True, xaxis_range=["2019-06-01 00:00:00", "2019-07-01 00:00:00"])